# Decorators

decorator는 다른 함수를 가져와서 그 함수를 수정하지 않고 기능을 확장하는 것
기존에 존재하는 함수에 새로운 기능을 부여하는 강력한 기능 

2가지 decorator가 존재 
* function decorator
* class decorator

@ 심볼을 사용하여 데코레이터 기능 수행 


# Function decorators

decorator는 다른 함수를 인자로 가져오고, inner function에서 새로운 기능의 동작 방식을 감싼다(wrap) 

아래 방법은 decorator를 사용하지 않는 방식  
함수에 다른 함수를 전달 

In [7]:
# A decorator function takes another function as argument, wraps its behaviour inside
# an inner function, and returns the wrapped function.
def start_end_decorator(func):
    
    def wrapper():
        print('Start')
        func()
        print('End')
    return wrapper

def print_name():
    print('Alex')
    
print_name()

print()

# Now wrap the function by passing it as argument to the decorator function
# and asign it to itself -> Our function has extended behaviour!
print_name = start_end_decorator(print_name)
print_name()

Alex

Start
Alex
End


In [ ]:
데코레이터로 위의 방식을 대체할 수 있음 

In [8]:
@start_end_decorator
def print_name():
    print('Alex')
    
print_name()

Start
Alex
End


## function arguments

만약에 함수에 인자가 포함되어 있다면 데코레이터 함수의 inner function에 \*args 와 **kwargs를 사용해야 함 

In [10]:
def start_end_decorator_2(func):
    
    def wrapper(*args, **kwargs):
        print('Start')
        func(*args, **kwargs)
        print('End')
    return wrapper

@start_end_decorator_2
def add_5(x):
    return x + 5

result = add_5(10)
print(result)

Start
End
None


## Return values

함수의 결과를 얻기 위해서 inner function(wrapper)에서 return을 정의해야 한다 

In [11]:
def start_end_decorator_2(func):
    
    def wrapper(*args, **kwargs):
        print('Start')
        result = func(*args, **kwargs)
        print('End')
        return result
    return wrapper

@start_end_decorator_2
def add_5(x):
    return x + 5

In [12]:
result = add_5(10)
print(result)

Start
End
15


## function identity

decorated function의 이름을 확인해보면 파이썬은 그 함수가 decorator function 안의 wrapped inner function이라고 생각 

In [13]:
print(add_5.__name__)
help(add_5)

wrapper
Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



이 문제를 고치기 위해 `functools.wraps`을 사용하면 original function의 정보를 유지할 수 있음 

In [14]:
import functools
def start_end_decorator_4(func):
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('Start')
        result = func(*args, **kwargs)
        print('End')
        return result
    return wrapper

@start_end_decorator_4
def add_5(x):
    return x + 5
result = add_5(10)
print(result)
print(add_5.__name__)
help(add_5)

Start
End
15
add_5
Help on function add_5 in module __main__:

add_5(x)



## final template for own decorators

In [15]:
import functools

def my_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # Do something before
        result = func(*args, **kwargs)
        # Do something after
        return result
    return wrapper

## decorator function arguments

decorator function에 argument를 전달하는 경우는 2개의 inner function을 사용해서 해결 할 수 있음 

`repeat`은 숫자 input을 받는다. 이 함수 내에서, `decorator_repeat`은 함수를 받고, `wrapper`에서 실제로 그 함수의 기능을 확장한다.

In [17]:
def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            for _ in range(num_times):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator_repeat

@repeat(num_times=3)
def greet(name):
    print(f"Hello {name}")
    
greet('Alex')

Hello Alex
Hello Alex
Hello Alex


## Nested Decorators

In [2]:
import functools

In [3]:
def start_end_decorator_4(func):
    
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('Start')
        result = func(*args, **kwargs)
        print('End')
        return result
    return wrapper

In [2]:
# a decorator function that prints debug information about the wrapped function
def debug(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        args_repr = [repr(a) for a in args]
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]
        signature = ", ".join(args_repr + kwargs_repr)
        print(f"Calling {func.__name__}({signature})")
        result = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {result!r}")
        return result
    return wrapper


# Class decorator

In [5]:
class CountCalls:
    # the init needs to have the func as argument and stores it
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0
    
    # extend functionality, execute function, and return the result
    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)

@CountCalls
def say_hello(num):
    print("Hello!")
    
say_hello(5)
say_hello(5)

Call 1 of 'say_hello'
Hello!
Call 2 of 'say_hello'
Hello!
